In [1]:
from bs4 import BeautifulSoup
import json
from urllib.parse import urljoin
import re
from collections import defaultdict
import requests

class Coletor:
    def __init__(self):
        self.urls_visitadas = set()
        self.dados_coletados = []

    def coletar_urls(self, url_inicial, profundidade=1):
        if profundidade == 0:
            return
        if url_inicial in self.urls_visitadas:
            return

        url_objeto = Url(url_inicial)
        html = url_objeto.buscar_html()
        if html:
            self.urls_visitadas.add(url_inicial)

            soup = BeautifulSoup(html, 'html.parser')
            for link in soup.find_all('a', href=True):
                href = link['href']
                href_absoluto = urljoin(url_inicial, href)
                if self.validar_url(href_absoluto):
                    self.dados_coletados.append({'url': href_absoluto})
                    self.coletar_urls(href_absoluto, profundidade - 1)

    def validar_url(self, url):
        return re.match(r'^https://www.ufop.br/', url)

    def salvar_dados_json(self, nome_arquivo):
        with open(nome_arquivo, 'w') as arquivo:
            json.dump(self.dados_coletados, arquivo, indent=4)

class Url:
    def __init__(self, url):
        self.url = url

    def buscar_html(self):
        try:
            resposta = requests.get(self.url, allow_redirects=True)
            if resposta.status_code == 200:
                return resposta.text
            else:
                print(f"Falha ao buscar o HTML de {self.url}. Código do erro: {resposta.status_code}")
                return None
        except requests.exceptions.RequestException as erro:
            print(f"Ocorreu um erro ao buscar o HTML de {self.url}: {str(erro)}")
            return None

class Indexador:
    def __init__(self):
        self.indice_invertido = defaultdict(list)

    def indexar_pagina(self, url, texto):
        soup = BeautifulSoup(texto, 'html.parser')
        publicacoes = [link.get_text() for link in soup.find_all('a', class_='author')]
        if publicacoes:
            self.indice_invertido[url].extend(publicacoes)
        else:
            print(f"Nenhuma publicação encontrada na página: {url}")

        if "coronavirus" in texto:
            print(f'Palavra "licenciamento ambiental" encontrada na página: {url}')

if __name__ == "__main__":
    urls = ["https://www.ufop.br/"]
    coletor = Coletor()
    indexador = Indexador()

    for url in urls:
        coletor.coletar_urls(url, profundidade=3)

    coletor.salvar_dados_json('dados_coletados.json')

    for url in coletor.urls_visitadas:
        html = Url(url).buscar_html()
        if html:
            indexador.indexar_pagina(url, html)

    while True:
        consulta = input("Digite sua consulta (ou 'sair' para encerrar): ")
        if consulta.lower() == 'sair':
            break


Nenhuma publicação encontrada na página: https://www.ufop.br/noticias/oportunidades/editais-de-bolsistas-e-voluntarios-com-inscricoes-abertas-0
Palavra "licenciamento ambiental" encontrada na página: https://www.ufop.br/noticias/oportunidades/editais-de-bolsistas-e-voluntarios-com-inscricoes-abertas-0
Nenhuma publicação encontrada na página: https://www.ufop.br/
Palavra "licenciamento ambiental" encontrada na página: https://www.ufop.br/
Nenhuma publicação encontrada na página: https://www.ufop.br/noticias/coronavirus/aplicativo-disponibiliza-questionario-de-autoavaliacao-para-covid-19
Palavra "licenciamento ambiental" encontrada na página: https://www.ufop.br/noticias/coronavirus/aplicativo-disponibiliza-questionario-de-autoavaliacao-para-covid-19
Nenhuma publicação encontrada na página: https://www.ufop.br/unidades-academicas
Palavra "licenciamento ambiental" encontrada na página: https://www.ufop.br/unidades-academicas
Nenhuma publicação encontrada na página: https://www.ufop.br/not